In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from skimage.transform import resize
import keras
import cv2
%matplotlib inline

import train_util
import util
from image_generator import Generator
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append('../keras-deeplab-v3-plus/')

/root/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from model import Deeplabv3
input_shape = (128, 128, 3)
#it looks like background class won't be counted
classes = 1
deeplab_model = Deeplabv3(input_shape=input_shape, classes=classes, backbone='xception')

In [4]:
deeplab_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 64, 64, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 64, 64, 32)   128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 64, 64, 32)   0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

## Prepare train val data generator

In [5]:
master_df = pd.read_csv('./data/depth_prop_cls_bad_mask.csv')
id_depth_df = master_df[['id', 'z']].set_index('id')['z']
depth_dict = id_depth_df.to_dict()
# data_type is 'train', 'test' and 'bad_train'
train_val_df = master_df[(master_df['data_type'] == 'train')]
ids = train_val_df['id'].values
classes = train_val_df['salt_propotion_class'].values
from sklearn.cross_validation import train_test_split
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=43, stratify=classes)
batch_size = 32
# not feature (depth) vec
tgs_generator = Generator(train_ids = train_ids, depth_dict=depth_dict, val_ids=val_ids, batch_size=batch_size, feature_out=False)

# steps_per_epoch
steps_per_epoch = len(train_ids) //  batch_size
validation_steps = len(val_ids) //  batch_size
# validation batch will be all validation
# validation_steps = 1
print(steps_per_epoch)
print(validation_steps)

97
24


## Train

In [ ]:
train_util.train(deeplab_model, 
                 tgs_generator.generate(True), 
                 tgs_generator.generate(False), 
                 metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef, train_util.mean_iou_threshold],
                 epochs=3000, steps_per_epoch=steps_per_epoch, 
                 validation_steps=validation_steps)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 64, 64, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 64, 64, 32)   128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 64, 64, 32)   0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

Epoch 1/3000
 - 101s - loss: 2.1400 - dice_coef: 0.3092 - binary_accuracy: 0.1178 - true_positive_rate: 0.6681 - iou_coef: 0.4355 - mean_iou_threshold: 0.3683 - val_loss: 2.7495 - val_dice_coef: 0.4628 - val_binary_accuracy: 0.0177 - val_true_positive_rate: -1.9724e+00 - val_iou_coef: 0.4808 - val_mean_iou_threshold: 0.4527
Finished epoch: 0
{'val_true_positive_rate': -1.9724084498981636, 'val_iou_coef': 0.48083115617434186, 'binary_accuracy': 0.11784435547504228, 'val_mean_iou_threshold': 0.4527343772351742, 'val_loss': 2.749477192759514, 'dice_coef': 0.30917676123454396, 'loss': 2.1399769549517287, 'iou_coef': 0.4355435738551248, 'val_binary_accuracy': 0.017692089080810547, 'mean_iou_threshold': 0.36826675323788655, 'val_dice_coef': 0.4627937798698743, 'true_positive_rate': 0.668070015622322}

Epoch 00001: val_loss improved from inf to 2.74948, saving model to ./log/2018_0811_0055/best_weights.hdf5
Epoch 2/3000
 - 85s - loss: 1.3187 - dice_coef: 0.3278 - binary_accuracy: 0.4149 - tru

Finished epoch: 9
{'val_true_positive_rate': -54.47092838585377, 'lr': 0.001, 'val_iou_coef': 0.45268194129069644, 'binary_accuracy': 0.7758570012358046, 'val_mean_iou_threshold': 0.3881510403007269, 'val_loss': 1.4351287210981052, 'dice_coef': 0.2265372322392218, 'loss': 0.48147693691179927, 'iou_coef': 0.3538684361067015, 'val_binary_accuracy': 0.4227445920308431, 'mean_iou_threshold': 0.2598260303901643, 'val_dice_coef': 0.30508782404164475, 'true_positive_rate': 0.5121637534292549}

Epoch 00010: val_loss did not improve
Epoch 11/3000
 - 85s - loss: 0.3669 - dice_coef: 0.2747 - binary_accuracy: 0.8267 - true_positive_rate: 0.6071 - iou_coef: 0.3562 - mean_iou_threshold: 0.2689 - val_loss: 0.5684 - val_dice_coef: 0.3354 - val_binary_accuracy: 0.6418 - val_true_positive_rate: -1.7266e+00 - val_iou_coef: 0.4355 - val_mean_iou_threshold: 0.3500
Finished epoch: 10
{'val_true_positive_rate': -1.72658639602984, 'lr': 0.001, 'val_iou_coef': 0.43548819422721863, 'binary_accuracy': 0.82670711

 - 85s - loss: 0.3968 - dice_coef: 0.2850 - binary_accuracy: 0.8505 - true_positive_rate: 0.6001 - iou_coef: 0.4622 - mean_iou_threshold: 0.3800 - val_loss: 0.4830 - val_dice_coef: 0.3270 - val_binary_accuracy: 0.7791 - val_true_positive_rate: -2.0489e-01 - val_iou_coef: 0.3381 - val_mean_iou_threshold: 0.2559
Finished epoch: 19
{'val_true_positive_rate': -0.2048945907736197, 'lr': 0.001, 'val_iou_coef': 0.33810753375291824, 'binary_accuracy': 0.850529719873802, 'val_mean_iou_threshold': 0.2558593787252903, 'val_loss': 0.4829920791089535, 'dice_coef': 0.2849860618409422, 'loss': 0.39675236224513694, 'iou_coef': 0.46219508641773893, 'val_binary_accuracy': 0.7790860335032145, 'mean_iou_threshold': 0.37999355608655006, 'val_dice_coef': 0.32696307947238284, 'true_positive_rate': 0.6001204001842085}

Epoch 00020: val_loss did not improve
Epoch 21/3000
 - 85s - loss: 0.7553 - dice_coef: 0.2545 - binary_accuracy: 0.8273 - true_positive_rate: 0.8627 - iou_coef: 0.3089 - mean_iou_threshold: 0.2

 - 85s - loss: 0.3570 - dice_coef: 0.2651 - binary_accuracy: 0.8667 - true_positive_rate: 0.6538 - iou_coef: 0.4709 - mean_iou_threshold: 0.3906 - val_loss: 0.4299 - val_dice_coef: 0.2830 - val_binary_accuracy: 0.8794 - val_true_positive_rate: 1.7048 - val_iou_coef: 0.5208 - val_mean_iou_threshold: 0.4292
Finished epoch: 29
{'val_true_positive_rate': 1.7047551708916824, 'lr': 0.00020000001, 'val_iou_coef': 0.520823589215676, 'binary_accuracy': 0.8666672460811654, 'val_mean_iou_threshold': 0.42916666716337204, 'val_loss': 0.4299011218051116, 'dice_coef': 0.26510012096043717, 'loss': 0.35698794043555704, 'iou_coef': 0.4708990288763931, 'val_binary_accuracy': 0.8794255256652832, 'mean_iou_threshold': 0.390625001382582, 'val_dice_coef': 0.2830106702943643, 'true_positive_rate': 0.6537626797390967}

Epoch 00030: val_loss did not improve
Epoch 31/3000
 - 85s - loss: 0.3700 - dice_coef: 0.2715 - binary_accuracy: 0.8711 - true_positive_rate: 0.7059 - iou_coef: 0.4495 - mean_iou_threshold: 0.36

...

 - 84s - loss: 0.3199 - dice_coef: 0.2831 - binary_accuracy: 0.8768 - true_positive_rate: 0.6964 - iou_coef: 0.4782 - mean_iou_threshold: 0.3972 - val_loss: 0.3950 - val_dice_coef: 0.2584 - val_binary_accuracy: 0.8821 - val_true_positive_rate: 5.6592 - val_iou_coef: 0.5153 - val_mean_iou_threshold: 0.4258
Finished epoch: 339
{'val_true_positive_rate': 5.659163435300191, 'lr': 4.294968e-26, 'val_iou_coef': 0.5152870317300161, 'binary_accuracy': 0.8767822698219535, 'val_mean_iou_threshold': 0.4257812487582366, 'val_loss': 0.39499584771692753, 'dice_coef': 0.2831338144762, 'loss': 0.3199480083799854, 'iou_coef': 0.47822972056791957, 'val_binary_accuracy': 0.8820931911468506, 'mean_iou_threshold': 0.3971649465179935, 'val_dice_coef': 0.2584266768147548, 'true_positive_rate': 0.6964291103721894}

Epoch 00340: val_loss did not improve
Epoch 341/3000
 - 85s - loss: 0.3252 - dice_coef: 0.2821 - binary_accuracy: 0.8745 - true_positive_rate: 0.6906 - iou_coef: 0.4817 - mean_iou_threshold: 0.3981

 - 85s - loss: 0.3288 - dice_coef: 0.2829 - binary_accuracy: 0.8721 - true_positive_rate: 0.6938 - iou_coef: 0.4809 - mean_iou_threshold: 0.4002 - val_loss: 0.3941 - val_dice_coef: 0.3028 - val_binary_accuracy: 0.8828 - val_true_positive_rate: 4.0284 - val_iou_coef: 0.5163 - val_mean_iou_threshold: 0.4270
Finished epoch: 349
{'val_true_positive_rate': 4.0283803554872675, 'lr': 8.589936e-27, 'val_iou_coef': 0.5163074322044849, 'binary_accuracy': 0.872101439643152, 'val_mean_iou_threshold': 0.4269531232615312, 'val_loss': 0.39409184207518894, 'dice_coef': 0.28293054128430556, 'loss': 0.32876625257669034, 'iou_coef': 0.4808778658355634, 'val_binary_accuracy': 0.8828320503234863, 'mean_iou_threshold': 0.4001610818290219, 'val_dice_coef': 0.30277152856191, 'true_positive_rate': 0.6938446867711765}

Epoch 00350: val_loss did not improve
Epoch 351/3000
 - 85s - loss: 0.3239 - dice_coef: 0.2798 - binary_accuracy: 0.8747 - true_positive_rate: 0.6895 - iou_coef: 0.4784 - mean_iou_threshold: 0.39

 - 85s - loss: 0.3240 - dice_coef: 0.2818 - binary_accuracy: 0.8750 - true_positive_rate: 0.6922 - iou_coef: 0.4808 - mean_iou_threshold: 0.3986 - val_loss: 0.4085 - val_dice_coef: 0.3013 - val_binary_accuracy: 0.8805 - val_true_positive_rate: 7.0204 - val_iou_coef: 0.5115 - val_mean_iou_threshold: 0.4223
Finished epoch: 359
{'val_true_positive_rate': 7.02037092546622, 'lr': 1.717987e-27, 'val_iou_coef': 0.5115443381170431, 'binary_accuracy': 0.8750106968830541, 'val_mean_iou_threshold': 0.4222656252483527, 'val_loss': 0.4085171427577734, 'dice_coef': 0.28184296981882806, 'loss': 0.3239796634801884, 'iou_coef': 0.48080794205985117, 'val_binary_accuracy': 0.8805161317189535, 'mean_iou_threshold': 0.398582475547938, 'val_dice_coef': 0.3013077725966771, 'true_positive_rate': 0.6922053559539244}

Epoch 00360: val_loss did not improve
Epoch 361/3000
 - 85s - loss: 0.3221 - dice_coef: 0.2832 - binary_accuracy: 0.8769 - true_positive_rate: 0.6945 - iou_coef: 0.4834 - mean_iou_threshold: 0.403

 - 85s - loss: 0.3254 - dice_coef: 0.2829 - binary_accuracy: 0.8758 - true_positive_rate: 0.6997 - iou_coef: 0.4790 - mean_iou_threshold: 0.3959 - val_loss: 0.4022 - val_dice_coef: 0.2678 - val_binary_accuracy: 0.8811 - val_true_positive_rate: 5.9349 - val_iou_coef: 0.5115 - val_mean_iou_threshold: 0.4224
Finished epoch: 369
{'val_true_positive_rate': 5.934860502680142, 'lr': 3.435974e-28, 'val_iou_coef': 0.5115115009248257, 'binary_accuracy': 0.8757877153219636, 'val_mean_iou_threshold': 0.42239583159486455, 'val_loss': 0.40219896224637824, 'dice_coef': 0.2828864553540023, 'loss': 0.32543026602145325, 'iou_coef': 0.4790332643027158, 'val_binary_accuracy': 0.8810848395029703, 'mean_iou_threshold': 0.3959085056462239, 'val_dice_coef': 0.26778766823311645, 'true_positive_rate': 0.6996627188834947}

Epoch 00370: val_loss did not improve
Epoch 371/3000
 - 85s - loss: 0.3197 - dice_coef: 0.2879 - binary_accuracy: 0.8776 - true_positive_rate: 0.7075 - iou_coef: 0.4844 - mean_iou_threshold: 0

 - 85s - loss: 0.3205 - dice_coef: 0.2853 - binary_accuracy: 0.8751 - true_positive_rate: 0.6897 - iou_coef: 0.4830 - mean_iou_threshold: 0.4005 - val_loss: 0.4123 - val_dice_coef: 0.2786 - val_binary_accuracy: 0.8786 - val_true_positive_rate: 7.0760 - val_iou_coef: 0.5158 - val_mean_iou_threshold: 0.4260
Finished epoch: 379
{'val_true_positive_rate': 7.075985733419657, 'lr': 6.871948e-29, 'val_iou_coef': 0.5157718571523825, 'binary_accuracy': 0.875064023991221, 'val_mean_iou_threshold': 0.4260416676600774, 'val_loss': 0.4123117153843244, 'dice_coef': 0.28527579633231015, 'loss': 0.32048100018009695, 'iou_coef': 0.48303571159077674, 'val_binary_accuracy': 0.8785665035247803, 'mean_iou_threshold': 0.4005154640096979, 'val_dice_coef': 0.27862431419392425, 'true_positive_rate': 0.6897141448001272}

Epoch 00380: val_loss did not improve
Epoch 381/3000
 - 85s - loss: 0.3228 - dice_coef: 0.2849 - binary_accuracy: 0.8749 - true_positive_rate: 0.7034 - iou_coef: 0.4659 - mean_iou_threshold: 0.

In [34]:
b = next(tgs_generator.generate(False))

In [39]:
b[1].shape

(32, 128, 128, 1)